https://github.com/fastai/courses/blob/master/deeplearning1/nbs/char-rnn.ipynb

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
import importlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import random
import time
import os
import pandas as pd
import csv
import math
import bcolz
import pickle
import re
import pathlib
import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

SOS_TOKEN = '<SOS>' # Start Of Word

# Data setup

In [3]:
# importing keras might cause problems with cudann version etc
# import keras # some good utils in here
# path = keras.utils.data_utils.get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")

In [4]:
path = '/home/ohu/.keras/datasets/nietzsche.txt'
text = open(path).read().lower()
path
print('corpus length:', len(text))

'/home/ohu/.keras/datasets/nietzsche.txt'

corpus length: 600893


In [5]:
!tail {path} -n5

not to be regarded as a pure type. Knowledge and science--as far as they
existed--and superiority to the rest of mankind by logical discipline
and training of the intellectual powers were insisted upon by the
Buddhists as essential to sanctity, just as they were denounced by the
christian world as the indications of sinfulness.

In [6]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
''.join(chars)

total chars: 57


'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxyzäæéë'

In [7]:
char_indices = dict((c, i) for i, c in enumerate(chars))
char_indices['\n']

0

# Utils

In [8]:
def string_to_tensor(in_str, chars_index=char_indices, as_variable=True):
    """Onehot encoded tensor of string"""
    tensor_length = len(in_str)
    tensor = torch.zeros(1, tensor_length, len(chars_index))
    for li, letter in enumerate(in_str):
        tensor[0, li, chars_index[letter]] = 1
    if as_variable:
        tensor = Variable(tensor).cuda()
    return tensor

string_to_tensor('hello', as_variable=False).size()

torch.Size([1, 5, 57])

In [9]:
def time_since(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

# Char Rnn

In [10]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CharRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.layers = 1
        
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.lin_output = nn.Linear(hidden_size, input_size)
        
        
    def forward(self, input_char_vs, hidden = None):
        if hidden is None:
            hidden = self.init_hidden()
        
        rnn_outputs, hidden = self.rnn(input_char_vs, hidden)
        
        output = self.lin_output(hidden)
        output = F.log_softmax(output[0])
       
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.layers, 1, self.hidden_size)).cuda()
    
model = CharRNN(len(chars), 512).cuda()
model

CharRNN (
  (rnn): GRU(57, 512, batch_first=True)
  (lin_output): Linear (512 -> 57)
)

In [11]:
#tmp = Variable(torch.FloatTensor([2]).view(1,1,-1)).cuda()
#tmp = Variable(string_to_tensor('hello')).cuda()
tmp = model(string_to_tensor('hello'))
len(tmp[0][0])
tmp[1].size()
#chars[tmp[0].topk(1)[1].data[0][0]]

57

torch.Size([1, 1, 512])

In [12]:
tmp[0][0].topk(1)[1].data[0]

8

# Eval

In [13]:
def print_example(iters=320, choice=True):
    seed_string="ethics is a basic foundation of all that"
    for i in range(iters):
        #x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        x = string_to_tensor(seed_string)
        output, hidden = model(x)
        if choice:
            next_char = np.random.choice(chars, p=F.softmax(output)[0].data.cpu().numpy())
        else:
            next_char_idx = output.topk(1)[1].data[0][0]
            next_char = chars[next_char_idx]
        # return next_char
        seed_string = seed_string + next_char
    print(seed_string)
#tmp = print_example(choice=False)
tmp = print_example(iters=40, choice=False)
tmp

ethics is a basic foundation of all that----------------------------------------


# Samples

In [22]:
def get_random_sample_string(length=41):
    sample_place = random.randint(0, len(text)-length-1)
    sample = text[sample_place:sample_place+length]
    return sample
get_random_sample_string()

'se rules them with the mastery of command'

In [24]:
sample_size = 50000

sample_sentence_size = 40
sample_data = torch.zeros(sample_size, sample_sentence_size, len(chars)) # .cuda()
#sample_target = torch.zeros((sample_size, sample_sentence_size, 1), torch.LongTensor)
sample_target = torch.LongTensor(sample_size, sample_sentence_size, 1).zero_() # .cuda()
for i in range(sample_size):
    sample = get_random_sample_string(sample_sentence_size+1)
    sample = [char_indices[c] for c in sample]
    for j in range(sample_sentence_size):
        sample_data[i][j][sample[j]] = 1
        sample_target[i][j] = sample[j+1]

In [25]:
i = 5
j = 10
sample_data[i].topk(1)[1][j][0] == sample_target[i][j-1][0]
''.join([chars[c[0]] for c in sample_data[i].topk(1)[1]])

True

'ets angry when the other man\nconclusivel'

In [26]:
clip_parameters_value = 0.25

def train_single(optimizer, loss_function, use_teacher_forcing):
    model.zero_grad()
    loss = 0
    
    sample_i = random.randint(0, len(sample_data)-1)
    x = sample_data[sample_i]
    y = sample_target[sample_i]
    
    input_char = x[0]
    target = Variable(y).cuda()
    
    output_line = []
    hidden = model.init_hidden()
    
    for i in range(len(x)):
        if use_teacher_forcing:
            input_char = x[i] # replace input with right target
        
        input_char = Variable(input_char).cuda().view(1,1,-1)
        output, hidden = model(input_char, hidden)
        loss += loss_function(output, target[i])

        char_idx = output.topk(1)[1].data[0][0]
        output_line.append(chars[char_idx])
        
        input_char = torch.zeros(1,1,len(chars))
        input_char[0,0,char_idx] = 1
            
    if clip_parameters_value:
        torch.nn.utils.clip_grad_norm(model.parameters(), clip_parameters_value)
    
    loss.backward()
    optimizer.step()

    return ''.join(output_line), loss.data[0] / len(sample)

#tmp = get_random_sample(20)
train_single(torch.optim.Adam(model.parameters(), lr=0.0001), nn.NLLLoss(), use_teacher_forcing=False)
train_single(torch.optim.Adam(model.parameters(), lr=0.0001), nn.NLLLoss(), use_teacher_forcing=True)
#tmp = train_single(tmp, torch.optim.Adam(model.parameters(), lr=0.0001), nn.NLLLoss(), use_teacher_forcing=True)

#tmp.topk(1)[1].data[0][0]

('ha,-------------------------------------', 3.945309150509718)

('h----a----aaa(a:aaaa-aa--aaa---------:--', 3.941068323647104)

In [27]:
def train_iterations(n_iters=100000, lr=0.001, teacher_forcing_ratio=0.5,
                     print_every=10000, plot_every=1000):
    global optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    start = time.time()
    model.train()

    current_loss = 0
    current_loss_iter = 0
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        result, loss = train_single(optimizer=optimizer, loss_function=nn.NLLLoss(),
                                    use_teacher_forcing=use_teacher_forcing)
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            teacher_forcing_str = ""
            if use_teacher_forcing:
                teacher_forcing_str = "(forcing)"
            #correct = '✓' if result == s_string else "✗: {}".format(s_string)
            
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} {}".format(
                      model_training.iterations, iteration/n_iters, time_since(start),
                      current_loss/current_loss_iter, loss, result, teacher_forcing_str))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 0
            
        #if model_training.iterations % 50000 == 0 or model_training.iterations == 10:
        #    model_training.save_models()
        #    acc = test_model_accuracy(encoder_rnn, test_model_single_sample)
        #    model_training.accuracy.append(acc)
    
    # test_model_accuracy(model, n_sample=10000)

In [28]:
from pytorch_utils_oh_3 import ModelTraining
MODEL_SAVE_PATH = 'char_rnn_fast_ai_testing_2'

In [29]:
model_training = ModelTraining(MODEL_SAVE_PATH, [model])

Save path: data/models/char_rnn_fast_ai_testing_2


In [30]:
train_iterations(n_iters=10, print_every=5)

     5  50% (   0m 0s)   3.870   |   3.80: r                                        
    10 100% (   0m 0s)   3.680   |   3.17: a                                        


### WITHOUT TEACHER FORCING

In [31]:
model = CharRNN(len(chars), 512).cuda()
model_training = ModelTraining(MODEL_SAVE_PATH, [model])

Save path: data/models/char_rnn_fast_ai_testing_2


In [32]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0)

  1000  10% (  0m 59s)   2.971   |   2.88:                                          
  2000  20% (  1m 55s)   2.953   |   2.98: e                                        
  3000  30% (  2m 52s)   2.952   |   3.25:  e                                       
  4000  40% (  3m 48s)   2.942   |   2.94: n                                        
  5000  50% (  4m 45s)   2.936   |   2.70: n                                        
  6000  60% (  5m 42s)   2.944   |   2.82: e                                        
  7000  70% (  6m 39s)   2.938   |   2.95: the                                      
  8000  80% (  7m 36s)   2.920   |   2.84:                                          
  9000  90% (  8m 33s)   2.932   |   2.84: l                                        
 10000 100% (  9m 30s)   2.933   |   2.82: to                                       


In [33]:
print_example()

ethics is a basic foundation of all thatnmor rterowt hile wincsoavrhenc eem .. onnts o tla  enee, mahdly  thennt qeoy nloinh  ulan  l 
llva tmes ekl epulm un,y , ovnchee  n sute nitnl a.  os  w
 r iten  whinelns =s a ptnn ioa ,
 imlntcs  elintes y ooneoul alonthel , is
eve .   sa tn a b f to ef spish i cneod geen
imodirss e ewh
ri indlsd nd o s th  iou llis 


In [34]:
print_example()

ethics is a basic foundation of all that reomrancreiasteol
 ,il ito o, ennetrde ate inncedyy  -
gn od, uhyubdimlul iefe,alnwtanily  iusres mnd tsiptss   oanpi hseudhere, iol ewfri xprns  n teun,  aul
os.ch a vel
eun s uom es  ; wh dy hs ore t ve; e ld w  einnoy anntunanelsunalllec n uldy ie oofuy   .   bo tountr u terwo ys, a ens n rd whmesinr 1lelen ics. ie


In [35]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0)

 11000  10% (  0m 56s)   2.927   |   3.01:   e                                      
 12000  20% (  1m 54s)   2.933   |   2.81:                                          
 13000  30% (  2m 51s)   2.944   |   2.85:                                          
 14000  40% (  3m 47s)   2.933   |   2.89: ee                                       
 15000  50% (  4m 44s)   2.931   |   2.85: r  an                                    
 16000  60% (  5m 41s)   2.931   |   2.89: th                                       
 17000  70% (  6m 38s)   2.950   |   2.60:  th                                      
 18000  80% (  7m 34s)   2.934   |   2.97: the                                      
 19000  90% (  8m 31s)   2.940   |   3.19: to                                       
 20000 100% (  9m 28s)   2.930   |   2.95: th                                       


In [36]:
print_example()

ethics is a basic foundation of all that allleths
ate
sseth, ien . u prxh e pee he t ncl  edhe n ogreuant is aes tenta 
, rhi o ghs escowo th  re yf iebitg tred y ,  rhemao  n  one.
sf,
npa  edry nef p eom ny, iirnte d: abl s enpacsndey ufset , tf ily ty ubtu teo sthy
wrcn prl" erwek d gintheu vely olchh n agh b one  el ara
npleeryeotwa ngahy, ot    nge wal 


In [37]:
print_example()

ethics is a basic foundation of all thatey topna   to to  mlef iutlesed s  tolly son: tne etu
bu n  nt l , auaicc suat
s l ey
drnd' b.eeoast anscenhnerlyo t, y aew tr i hestr    orharsheey hroat sl,o s eses
beltrie ss stmal ions e  alo weery neinisela, gae h  sa tteks  itt kh 
asin  ,s marlec o  ihs isth  s
n 
"th he - toenr
paf  s nertacs irten 
e sea enlr 


In [38]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0)

 21000  10% (  0m 57s)   2.931   |   3.20: n                                        
 22000  20% (  1m 54s)   2.937   |   2.88: e                                        
 23000  30% (  2m 51s)   2.926   |   3.11:  t                                       
 24000  40% (  3m 47s)   2.940   |   3.29: e                                        
 25000  50% (  4m 44s)   2.928   |   2.96: st                                       
 26000  60% (  5m 41s)   2.921   |   3.20: orie                                     
 27000  70% (  6m 38s)   2.938   |   2.69:                                          
 28000  80% (  7m 35s)   2.921   |   3.07:                                          
 29000  90% (  8m 32s)   2.927   |   2.77: h te                                     
 30000 100% (  9m 29s)   2.942   |   2.80: ng                                       


In [39]:
print_example()

ethics is a basic foundation of all thaty vhrke v s, y wo  hady   epidionef te a, bohe rei  ctit, n9 g che.  cy aocssm iaminr  stres as foc eetr hourong ng  s  ido nve exestrase plf woler dainnsont sr   ves iein  fan e innst  ehotfeav thn ue tn rher nat ni
tdinl  nonnethauanree sits y nf, igrghom s n pntlit loher aesary s orina,  eesar: t ciouni, toca um ee 


In [40]:
print_example()

ethics is a basic foundation of all thatyin el.
 nd neas gy fone  : a burniy t s :o ao seawnert eesin  hicntis  haneisn w an  oreeof"s  nnoe ths st oseian
ingss s e  er   ase hdulhs , woprnspar,  ad t ighen anoe  wh atha ist eg, ast res icit oum  oilssni   i ns o s ntejhayto th sensan teacenmaan,s f ptr n  y , hene n . ne sn.  iow  ndrt bley ts s, wuisi euei


In [41]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0)

 31000  10% (  0m 57s)   2.932   |   3.13:                                          
 32000  20% (  1m 54s)   2.929   |   2.84: ee                                       
 33000  30% (  2m 51s)   2.929   |   2.81: n                                        
 34000  40% (  3m 48s)   2.923   |   2.86: e                                        
 35000  50% (  4m 46s)   2.921   |   2.99: n                                        
 36000  60% (  5m 43s)   2.926   |   2.84: n                                        
 37000  70% (  6m 41s)   2.924   |   2.62:  ah                                      
 38000  80% (  7m 38s)   2.935   |   2.80: e t                                      
 39000  90% (  8m 36s)   2.933   |   2.92: t                                        
 40000 100% (  9m 33s)   2.927   |   2.74:   e                                      


In [42]:
print_example()

ethics is a basic foundation of all that h e anndhekndum, hotek
d tot
onnd als," ad se  coloan jesifnal, hadndt d yr ssot ls nterzlolrfa nliace r woteroan cioto, tvjunbll , so l h gg cil  ?s qniee  feele
  hien emenridendt hn,-te  ly   e o hran, ahehtane nn
gisnrat
etered  aotr, nusow cnitldchmtirwaons i eode gthc iccins nds p ab y ehiel na s a lw hn e
   v



In [43]:
print_example()

ethics is a basic foundation of all thate
disnce ce1oiaf
 omt sheu
dan 

ictuos  ohos ieis ss fe.
teearleel peanf d, oton s eyoif civac or, aoagslet efa tole, iapionca e mst meo e oallin y lilit ,eing  aes eiwoanewnr. ioms eanid a dthar , a iomhemeaa we hcie innweih.itnndt en t cgummendrdt, l es  o ad? arromem
eeut   ec nainst sth ro  dandeaeui he e hsusrvew


In [44]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0)

 41000  10% (  0m 57s)   2.926   |   2.74:  se                                      
 42000  20% (  1m 54s)   2.930   |   2.71: n                                        
 43000  30% (  2m 51s)   2.927   |   2.94:   o                                      
 44000  40% (  3m 48s)   2.926   |   3.26:  te                                      
 45000  50% (  4m 45s)   2.930   |   2.82: t                                        
 46000  60% (  5m 43s)   2.927   |   2.95: or                                       
 47000  70% (  6m 40s)   2.927   |   2.83:  t                                       
 48000  80% (  7m 37s)   2.931   |   2.68: th                                       
 49000  90% (  8m 35s)   2.923   |   2.86: th                                       
 50000 100% (  9m 32s)   2.932   |   2.89:  to                                      


In [45]:
print_example()

ethics is a basic foundation of all thatsasctioniman, ain
c nc
buo iost
il.h tht wot whte, woldy"f  tmetfou  ele t g
--tha   n
chat ofreuhe towmal  uiceanle ios  fo  t iviblc ainineliklotseho
biur thedg  t
to eluman "pord nuar t
y  ues ?r arigy raenod, afeg ln did
ts t ep  inl , aa  cspruocn  rer rurtmdanard 
hfinnn 
ina ema wiuntin thin ocen an  them nse no


In [46]:
print_example()

ethics is a basic foundation of all thatprretsunncen e tf hiss-- as i
 eeb eelatthni ly, ail ts m.anf otet uhtan thet sase
 -om g n u ct do aelt tivvea  oe ooltan whrclin   c lethenocen ln f-aietn eriv ndo y  coit hhe enode, ao oteh, bamerool  nl g    ,heloli  vey ahime u ceob s deel neoob crse s, phr   ta ven
ens tceld nenlion o  anitinvens
 isr lt f ceundi


In [47]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0)

 51000  10% (  0m 57s)   2.931   |   3.02:                                          
 52000  20% (  1m 54s)   2.930   |   2.98: t                                        
 53000  30% (  2m 51s)   2.933   |   2.96: n                                        
 54000  40% (  3m 47s)   2.932   |   2.80: n                                        
 55000  50% (  4m 44s)   2.935   |   3.10:                                          
 56000  60% (  5m 41s)   2.924   |   2.92: e                                        
 57000  70% (  6m 38s)   2.930   |   2.93: erte                                     
 58000  80% (  7m 35s)   2.931   |   2.83:   f                                      
 59000  90% (  8m 31s)   2.924   |   2.81:                                          
 60000 100% (  9m 28s)   2.928   |   2.95:                                          


In [48]:
print_example()

ethics is a basic foundation of all thatne anforl-tiat f  ruioln r esatuslats lior  petet ly e tars n  
hu ove
hpsece art mamiet )m s eempit th  uand, y ie ge ten bliermagedat r onif ictaese  o
rnn ofn inieg (eh-ei ulnnc g fiene, taehrlr re rroanedhoveleo meotalof tns ifeonmpesh son hurteun--h, i l bleeruristhrte tay o n ei n= ,ntof nce t ioot thb  tuie nde.


In [49]:
print_example()

ethics is a basic foundation of all thats  sulie as
uastore a gd  beiuriiee aheuen th cie fe, aglis ighine-  hanmremut,on ra ern,--hy nm f s oiey, 

ufonghe perhesd ooe foot.
 lo mntat de th dasd-ufs
ta  t mfech nt se ims tilis tcee
,  no f t e   t org.  teth, sori el w=re arady  hiovematbe cny h tiane, reac! p chi, mtsniete  eacisign asntoeavecon tletm eire


In [50]:
print_example()

ethics is a basic foundation of all thatin t iith y fhumin ar rig
cte eha c alemeg mno  aheeiamveelo  osk ner nictl ens adenl teelyvein d tpevenged ena d  uln ut n rdes hhei scenaniuuc to n dtrt  h od wnc eesdesehes.


t

oveealesilimbend --hint ttrem t oeit s  t ie iase ssthb oere i-o ast ses-seilsunirent han,  y
notton piaain  eaestu  wh, as  ! iatigen nan


In [51]:
print_example()

ethics is a basic foundation of all thattile  
te bl  ew du prt s onie  s mrnh  reanu    pitainisnce. , y pen y en n n s onwt-ha e serm shal ." -nirmeue iyn  esfin hirisn
 telonhidtlse sl ked r ha el nest hal, tiuniveosng o eventhv diieepesieo sesiss sre ssl nd t  t li
tuote?ger peei , heiedr  matatngag sea lsene5. raml h eer baeei sto  al ielisleard dth tsu


### WITH TEACHER FORCING

In [52]:
model = CharRNN(len(chars), 512).cuda()
model_training = ModelTraining(MODEL_SAVE_PATH, [model])

Save path: data/models/char_rnn_fast_ai_testing_2


In [53]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0.5)

  1000  10% (  0m 57s)   2.827   |   2.87: e  e  e  e  e  e  e  e  e  e  e  e  e  e 
  2000  20% (  1m 54s)   2.637   |   2.00: tere   and tertdtest ree e  the   ereahe (forcing)
  3000  30% (  2m 51s)   2.569   |   3.07:                                          
  4000  40% (  3m 48s)   2.513   |   3.61: ent of the ereat is and the ereat is and 
  5000  50% (  4m 45s)   2.462   |   2.31: nl aroet n ng  ausg yng  aor  ne ahrd    (forcing)
  6000  60% (  5m 42s)   2.417   |   2.38:    ng    arong tonstircey anprii wheenee (forcing)
  7000  70% (  6m 40s)   2.384   |   1.60: st  tn t dtn tiunant oor the fs ioenetio (forcing)
  8000  80% (  7m 37s)   2.378   |   1.82: ng ech a da d irs torvle tile ae onesooe (forcing)
  9000  90% (  8m 34s)   2.325   |   2.01:  y    aroceselibidan erpoion altetecal a (forcing)
 10000 100% (  9m 31s)   2.356   |   1.85:  er  arsti  -t  uirlir  aas anlays aowd  (forcing)


In [54]:
print_example()

ethics is a basic foundation of all that they wap in thifk, be nomen, that which
ict gessing be mustly almost
fises --or an seo laurect. stmunct. in "the pustle stlpny in the ieflictarism a
domans once ass impulses "morns, oo to anslioves thinks it theorous masticitity hander upog to lich upuraress not  on
dold it is necomen a are andive gaents and sus moumi


In [55]:
print_example()

ethics is a basic foundation of all that souls that thes a morradr, duffelence wholegradege, in
thir secteat who has one itse-foter they knowly is it ounsersulsideces of hishess sure any not hcoulf castermy of his importtings rave recharical quastures himselves ow the godgacely allore worst tsakes not a beand
trours, thearther to the moril as a chaturies uso


In [56]:
train_iterations(n_iters=10000, print_every=1000, teacher_forcing_ratio=0.5)

 11000  10% (  0m 57s)   2.358   |   2.93: it                                       
 12000  20% (  1m 54s)   2.355   |   2.96:  te                                      
 13000  30% (  2m 51s)   2.289   |   3.12:  to                                      
 14000  40% (  3m 49s)   2.318   |   1.30: toteawness,and seilidity an thich sn ias (forcing)
 15000  50% (  4m 46s)   2.298   |   1.65: r tot  e n ainting tn tremte oor theoado (forcing)
 16000  60% (  5m 43s)   2.337   |   3.04:                                          
 17000  70% (  6m 40s)   2.280   |   3.21: stessing the e oist not of the           
 18000  80% (  7m 38s)   2.306   |   2.54:   or                                     
 19000  90% (  8m 35s)   2.247   |   1.61: ne trrple tneund tounghi kre tl t trrter (forcing)
 20000 100% (  9m 32s)   2.268   |   2.68: e  e                                     


In [57]:
print_example()

ethics is a basic foundation of all that. a vanitys.
having a tur's esthatmen of eventivence prened in here upent
aha"s and schopenherecition, a mose, cllisg, and ipessible any pretrouphed and the approtents of to-will,"--it is muntfrench intraves even
tomarhend pool thkinglys but around fairs
larbith of is a guidions, and opinion in thets. dinging with the 


In [58]:
print_example()

ethics is a basic foundation of all that
are procable is astente wish be instrupt, any ever this bitcer that he can be
to goen
usunced prong himself
tois ly
struggle, in faith, being buck foundatedintary as as unelittle outs, and ever, artical "e"_s." on the grouf therewishes of a somefitua-it just
is that ir in the known of here careblines, likes appropers



In [59]:
train_iterations(n_iters=30000, print_every=1000, teacher_forcing_ratio=0.2)

 21000   3% (  0m 57s)   2.677   |   3.08: n                                        
 22000   7% (  1m 54s)   2.675   |   1.35: sf telpotnd sonccmetion  toom ahes ioale (forcing)
 23000  10% (  2m 52s)   2.678   |   3.15:                                          
 24000  13% (  3m 48s)   2.699   |   2.82: e  e                                     
 25000  17% (  4m 45s)   2.704   |   3.02: andter an                                
 26000  20% (  5m 42s)   2.653   |   2.73: et                                       
 27000  23% (  6m 38s)   2.681   |   2.91: h                                        
 28000  27% (  7m 35s)   2.658   |   2.70: r the                                    
 29000  30% (  8m 31s)   2.668   |   2.00: sete ond tor     siof t e  esconteog ais (forcing)
 30000  33% (  9m 28s)   2.675   |   2.81: ng oe                                    
 31000  37% ( 10m 25s)   2.665   |   1.39: nn d tn tvrope  an torhaps ooe seli ontl (forcing)
 32000  40% ( 11m 21s)   2.691   |   1

In [60]:
print_example()

ethics is a basic foundation of all that
calaral it;--it is of a good,
they,
woman forthing
at alsiking the spirral civilizaties. we approded citious like with mofinoly cal has aried in
these perpotity that is perpiest nainity but? as a prite naiused, it is idous butgredity in acimation he what
only them are slate," it is not fortorew order upon which had sa


In [61]:
print_example()

ethics is a basic foundation of all that there are place "facting macksion "friend (or everything hald a nigural
isterficied" man;--that thisieves throush this woll
begrught in the ininglengeness of unexumplation  pising ageing relaged; and its
dowands that it make shave an
ond call with them is are more
plibess this this
becime, with fools its
supposticing 


In [62]:
train_iterations(n_iters=30000, print_every=1000, teacher_forcing_ratio=0.1)

 51000   3% (  0m 56s)   2.799   |   1.20: ane tnmesponction oe aeen taralitnd pnpe (forcing)
 52000   7% (  1m 52s)   2.810   |   4.19: tnd regards the sthe e of the power of t 
 53000  10% (  2m 48s)   2.809   |   2.77: tne                                      
 54000  13% (  3m 44s)   2.817   |   2.86: e                                        
 55000  17% (  4m 40s)   2.781   |   2.81:                                          
 56000  20% (  5m 35s)   2.806   |   3.26: -ht the                                  
 57000  23% (  6m 31s)   2.797   |   2.98: n                                        
 58000  27% (  7m 27s)   2.816   |   2.83:   at                                     
 59000  30% (  8m 23s)   2.806   |   3.06:  wh                                      
 60000  33% (  9m 19s)   2.803   |   3.02: f the                                    
 61000  37% ( 10m 15s)   2.795   |   3.03:                                          
 62000  40% ( 11m 11s)   2.794   |   3.05:  en          

In [63]:
print_example()

ethics is a basic foundation of all that as  eubility anndeling have
he only encein one has noight the nee friey almost incrabacity and parking to the sti rffy a tot comous hethe men, and has tos of the so mode, in  in those who divind dusticicutebleness forces firs fas, analys of suffirical to the say, than ofly whournhey dancemough  ts ill, all recountingl


In [64]:
print_example()

ethics is a basic foundation of all that we resitnes, of the ruling there is nothintcy founces at ill, and witcherelves stoul comu atolicate, as in exception of a hands thaw action which withblide be wthe overcome, inthisherstication.
the si, made. what then will away! ymot repunses. reald whose sacrifice becausand booking of readens assver, in the presesube


In [65]:
train_iterations(n_iters=30000, print_every=1000, teacher_forcing_ratio=0.0)

 81000   3% (  0m 55s)   2.934   |   2.83: er                                       
 82000   7% (  1m 51s)   2.930   |   2.85: he                                       
 83000  10% (  2m 47s)   2.934   |   3.00: he                                       
 84000  13% (  3m 43s)   2.931   |   3.00:  
to                                     
 85000  17% (  4m 38s)   2.936   |   3.02:                                          
 86000  20% (  5m 34s)   2.925   |   2.95: th                                       
 87000  23% (  6m 29s)   2.920   |   3.36:                                          
 88000  27% (  7m 25s)   2.933   |   2.99: eete                                     
 89000  30% (  8m 20s)   2.938   |   2.85: e                                        
 90000  33% (  9m 16s)   2.921   |   3.01: hih                                      
 91000  37% ( 10m 12s)   2.935   |   2.80: o o                                      
 92000  40% (  11m 8s)   2.928   |   2.75: e e                   

In [66]:
print_example()

ethics is a basic foundation of all thatowerols fucp inosh pesu benig desewildehpetic nnoug
oeinpebe thdert fehemf maniennt  l se htio ptrsto eubl; nig handvell bachlyt e  encat; ine tme,
the nh an, yhim,, thone roe d ce forl, usginhin,--h thinnarang in of
hou ofd in wis dainit hraunin of,'s te, ruge ce be whhegan e et feampast ouc cinans dvely ii twahhe  an


In [67]:
print_example()

ethics is a basic foundation of all thative neg, ih msur,'
ore); ahiun
lofe, aaeurhermply takimathy
fo ebjonem natougerushiw eran warbud of rht an
es it cidmer ding,'deverokerealy fing unbe culwhupisfolle  hs la, ht shugious, inthencenst fogtham
hag siol mashe ge ube nish womorbes ins yest, aur te pho!
i omeagher dane weerscince hrece   allonl, fivel, wenmer


In [68]:
train_iterations(n_iters=30000, print_every=1000, teacher_forcing_ratio=0.0)

111000   3% (  0m 55s)   2.919   |   2.93:                                          
112000   7% (  1m 51s)   2.930   |   2.70:                                          
113000  10% (  2m 47s)   2.936   |   2.83:  aee                                     
114000  13% (  3m 42s)   2.931   |   2.98: e    e                                   
115000  17% (  4m 38s)   2.923   |   2.85:                                          
116000  20% (  5m 33s)   2.923   |   2.73:  an                                      
117000  23% (  6m 29s)   2.924   |   3.08:   e                                      
118000  27% (  7m 24s)   2.931   |   2.90: e                                        
119000  30% (  8m 20s)   2.929   |   2.81: to   e                                   
120000  33% (  9m 16s)   2.919   |   2.81:  tt e                                    
121000  37% ( 10m 11s)   2.923   |   3.20:  ah                                      
122000  40% (  11m 7s)   2.932   |   3.00:                       

In [69]:
print_example()

ethics is a basic foundation of all thats s d rhknoans -walressegarpeo
s  h r ytan,  eleso-ea d y ta sa midic  vorpanrr utaaiurhun.  o5ngefgerb to, fwro
aieo o slfew re  , sm
sempeerrtoeie -, t o sequagore rnet ltss hioit o eahsity jurzoiouel  vut. n  oisur ighro io theeicteaiale f neno-
 itod i gieh onovret ons   c
na eh i nf cumdnntneoraldy ttsenade eun. t


In [70]:
print_example()

ethics is a basic foundation of all thatl y--ialased, ahefrodes
eu eofrapilul,
becas san rhimmon,n st  dnstalihlfesf t eoher--sssthelcheeo
nillrnecos a ms chdbilamis sur, sneveryura iect mnirf it be riseilnisaldnof ise  ileanwhutrry
the  t uthth lt. i icar  anhee hemesnef t s fitchens er be.tim sod tpyrntth-oeityrine, lnd  ng, atgspsthen, uehg wet as  uthret


In [71]:
print_example()

ethics is a basic foundation of all thatg  ctot ehldue. whiins n tew r-thaates dleigigefoe s ohu
eedenfthegat sten?- e t owiest tmonteee aopie a twht,at
i goca,
e21re ven il, as t einteeutinbede:g--h ie tit  oo  st canoiote thet bo e d  ual sast e he wht uardel nesato  ihs, ettyw  nhbhe "amenhrtan e  is of
fr  kns  rneohmehanens o sef th nwaohe ts fr   efre 


In [72]:
print_example()

ethics is a basic foundation of all thatsandory n  t oielee, ttsaei f n pmirto  mivaio bario, ss asecloofal, devy re.icnt omesoheneinwoear
e  beevtu w se ee
 oe g
=ax mas n ileltoem  cmealny odritee-, -l-se  sil, e bein hn: oa eat,werthwhin n bownece ndanitel arish, hed vo oiess datpornsm uh hives a- t usbe sreje  r lllalt g it ch, le hia radf chever ionn  h
